<a href="https://colab.research.google.com/github/agprasanna/Coursera_Capstone/blob/master/Opening_an_Indian_Restaurant_in_Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install the necesary packages (if not done already)


In [3]:
#!conda install -c conda-forge geopy --yes 
#!conda install -c conda-forge folium=0.10.0 
#!conda install -c conda-forge geopandas
!pip install geopandas

     |████████████████████████████████| 972kB 2.7MB/s 
     |████████████████████████████████| 14.7MB 298kB/s 
     |████████████████████████████████| 10.9MB 45.2MB/s 


Import the necesary libraries here


In [4]:
import pandas as pd
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

import numpy as np

import urllib.request
import requests

from geopy.geocoders import Nominatim    # import geocoder Nominatim
import folium                            # import map rendering 

import geopandas as gpd             #to read in shape file and provides high #level interface with #matplotlib library for making maps

import matplotlib.cm as cm          # Matplotlib and associated plotting modules
import matplotlib.colors as colors 

#import k-means from clustering stage
from sklearn.cluster import KMeans

print("libraries imported!")

libraries imported!


Importing the geojson file from the OpenData project of City of  Toronto

In [5]:
Toronto_geojson_url= "https://ckan0.cf.opendata.inter.prod-toronto.ca/download_resource/a083c865-6d60-4d1d-b6c6-b0c8a85f9c15?format=geojson&projection=4326"

Toronto_Geojson = gpd.read_file(Toronto_geojson_url)
Toronto_Geojson.head(2)

,_id,AREA_ID,AREA_ATTR_ID,PARENT_AREA_ID,AREA_SHORT_CODE,AREA_LONG_CODE,AREA_NAME,AREA_DESC,X,Y,LONGITUDE,LATITUDE,OBJECTID,Shape__Area,Shape__Length,geometry
0,6581,25886861,25926662,49885,94,94,Wychwood (94),Wychwood (94),None,None,-79.425515,43.676919,16491505,3.217960e+06,7515.779658,"POLYGON ((-79.43592 43.68015, -79.43492 43.680..."
1,6582,25886820,25926663,49885,100,100,Yonge-Eglinton (100),Yonge-Eglinton (100),None,None,-79.403590,43.704689,16491521,3.160334e+06,7872.021074,"POLYGON ((-79.41096 43.70408, -79.40962 43.704..."


Extracting only the relevant information from the above dataset

In [6]:
Toronto_Geojson1 = Toronto_Geojson[['AREA_SHORT_CODE','geometry']]
#Toronto_Geojson1.head()

Another way to get the neighborhood names and geocoding


In [8]:
url = "https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Toronto#Table"
table_list = pd.read_html(url)
#print("number of tables in web page = ", len(table_list))
Raw_df = table_list[11] # what we want is the 11th table in the list

#Raw_df.head()
Raw_df.drop(columns=['Map', 'Unnamed: 5'], axis=1, inplace = True)
Neighbor_df = Raw_df.rename(columns={'CDN number': 'Code', 
                                     'City-designated neighbourhood':'Neighborhood_Name', 
                                     'Former city/borough':'Borough',
                                     'Neighbourhoods covered':'Areas_Covered'
                                    })

# Explode/Split Neighbourhood column into multiple rows (removing the hyphen symbol)
new_df = pd.DataFrame(Neighbor_df.Neighborhood_Name.str.split('-').tolist(), index=Neighbor_df.Code).stack()
new_df = new_df.reset_index([0, 'Code'])
new_df['Lat'] = np.NaN
new_df['Long'] = np.NaN
new_df.columns = ['Code', 'Neighborhood_Name', 'Lat', 'Long']

#### Geo coding the longitude and lattitude values using Nominatim ###

from functools import partial
#from geopy.extra.rate_limiter import RateLimiter

geolocator = Nominatim(user_agent="Neighborhood_explorer")

_geocode = partial(geolocator.geocode, language="en")
geocode = lambda query, **kw: _geocode("%s, Toronto Ontario Canada" % query, **kw)
#geocode = RateLimiter(geocode_, min_delay_seconds=1)

def eval_lat(x):
    try:
        return (x.latitude)
    except:
        return (None)
def eval_lon(x):
    try:
        return (x.longitude)
    except:
        return (None)

    
new_df['Lat'] = new_df['Neighborhood_Name'].apply(geocode).apply(lambda x: eval_lat(x))
new_df['Long'] = new_df['Neighborhood_Name'].apply(geocode).apply(lambda x: eval_lon(x))

print("Geo Coding complete!")
new_df



Geo Coding complete!


,Code,Neighborhood_Name,Lat,Long
0,129,Agincourt North,43.808038,-79.266439
1,128,Agincourt South,43.785353,-79.278549
2,128,Malvern West,43.790679,-79.248491
3,20,Alderwood,43.601717,-79.545232
4,95,Annex,43.670338,-79.407117
...,...,...,...,...
201,97,Yonge,43.761452,-79.410915
202,97,St.Clair,43.687951,-79.393695
203,27,York University Heights,43.758781,-79.519434
204,31,Yorkdale,43.724642,-79.447503


In [11]:
#save the Geocded Datafrem to a CSV for future usage.
new_df.to_csv('TorontoNeighborhoodGeocoding.csv', index = False)
#Import the geocode data from csv file 
new_df = pd.read_csv("TorontoNeighborhoodGeocoding.csv")

In [13]:
# find the neighborhoods that dont have lat,long values and drop them.
# Then remove the duplicates (keeping the first record)from "Code"
new_df[new_df.isna().any(axis=1)]
new_df.dropna(axis=0, inplace=True)
print("shape before removing duplicates: ",new_df.shape)
new_df.drop_duplicates(subset ="Code", keep = 'first', inplace = True) 
print("shape after removing duplicates: ",new_df.shape)

shape before removing duplicates:  (195, 4)
shape after removing duplicates:  (136, 4)


In [29]:
##Determining the centre for the map (I have tried two ways as mentioned below)

# Assigning centroids to map using the center point of the geojson data
#latitude= Toronto_Geojson.centroid.x.mean()
#longitude=Toronto_Geojson.centroid.y.mean()

# Assigning a selected location in Toronto as the center point, using geocoding
address = 'Glenview senior public school, Ontario, Canada'
geolocator = Nominatim(user_agent="Toronto_map")
MapCenterLocation = geolocator.geocode(address)
MapCenterLatitude = MapCenterLocation.latitude
MmapCenterLongitude = MapCenterLocation.longitude
#print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

In [15]:
Toronto_df = Toronto_Geojson[['AREA_SHORT_CODE','AREA_NAME','LONGITUDE','LATITUDE']]
#rename columns for convenience 
Toronto_df = Toronto_df.rename(columns={'AREA_SHORT_CODE': 'Code', 'AREA_NAME': 'Neighborhood', 'LONGITUDE': 'Longitude', 'LATITUDE': 'Latitude'})
Toronto_df['Neighborhood'] = Toronto_df['Neighborhood'].str.partition('(')
Toronto_df.head()

,Code,Neighborhood,Longitude,Latitude
0,94,Wychwood,-79.425515,43.676919
1,100,Yonge-Eglinton,-79.403590,43.704689
2,97,Yonge-St.Clair,-79.397871,43.687859
3,27,York University Heights,-79.488883,43.765736
4,31,Yorkdale-Glen Park,-79.457108,43.714672


In [16]:
Toronto_df.dtypes


Code              int64
Neighborhood     object
Longitude       float64
Latitude        float64
dtype: object

In [30]:
# create map of Toronto using latitude and longitude values created above
map_Toronto = folium.Map(location=[MapCenterLatitude, MmapCenterLongitude], 
                         zoom_start=11,
                         width='80%',
                         height='80%', 
                         min_zoom=10,
                         max_bounds= True, 
                         min_lat=43.5,
                         max_lat=44,
                         min_lon=-79.8,
                         max_lon=-78.9)

## add markers to map using Toronto_df 
for lat, lng, neighborhood in zip(Toronto_df['Latitude'], Toronto_df['Longitude'], Toronto_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        #fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)

## add markers to map using new_df
for lat, lng, neighborhood in zip(new_df['Lat'], new_df['Long'], new_df['Neighborhood_Name']):
    label = '{}, {}, {}'.format(neighborhood,lat,lng)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='red',
        fill=True,
        #fill_color='#3568cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)

## add neighborhood boundaries to map using TorontoGeojson data
folium.GeoJson(Toronto_Geojson, 
               name='Neighborhoods',
               ).add_to(map_Toronto)

folium.LayerControl().add_to(map_Toronto)



map_Toronto

## Selecting the neighborhood center point 
<p> As seen from the above map, there is a significant variation of the results (for selecting the center point for a neighborhood) when using the Nominatiam geocoding method and using the centerpoid of the GeoJson data. 

Nominatiam did not return all the geocodes for all neighborhoods. Further, it does not provide an even spread between the point in some cases. i.e. two points of two neighborhoods can be very close to each other resulting in poor venue data later on.

Considering the above shortcomings, I will use the centeroid of each neighborhood geometry ploygon, as the location (lat,long) for vanue data collection.

